# VCI Bayes Explore

## 1. Environment Setup

Import dependencies, configure plotting defaults, and align pyAgrum's visual output.

In [ ]:
# Core Python utilities
import base64
import io
import os
import random
from itertools import product
from pathlib import Path

# Numerical and data processing
import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency

# Visualisation helpers
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from IPython.display import display
from ipywidgets import interact, fixed

# Bayesian network tooling (pyAgrum ecosystem)
import pyagrum as gum
import pyagrum.lib.explain as explain
import pyagrum.lib.explain as expl
import pyagrum.lib.notebook as gnb
import pyagrum.lib.utils as gutils
import pyagrum.lib.bn2graph as gumb2g
import pyagrum.lib.bn_vs_bn as bnvsbn
import pyagrum.lib.bn_vs_bn as gcm
from pyagrum import BNLearner
from pyagrum.lib.bn2roc import showROC
from pyagrum.lib.discreteTypeProcessor import DiscreteTypeProcessor
import pyagrum.skbn as skbn
from pyagrum.skbn import BNClassifier

# Scikit-learn helpers
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import warnings

# Matplotlib defaults for manuscript-ready figures
warnings.filterwarnings('ignore', category=UserWarning)

mpl.rcParams['font.family'] = 'Helvetica Neue'
mpl.rcParams['font.size'] = 14
mpl.rcParams['font.weight'] = 'regular'


In [ ]:
# Utility: format dictionaries for side-by-side HTML display in the notebook
def dict2html(di1, di2=None):
  res = "<br/>".join([f"<b>{k:15}</b>:{v}" for k, v in di1.items()])
  if di2 is not None:
    res += "<br/><br/>"
    res += "<br/>".join([f"<b>{k:15}</b>:{v}" for k, v in di2.items()])
  return res


In [ ]:
# Ensure pyAgrum renders graphs with consistent layouts across reruns
gum.config["notebook", "graph_layout"] = "dot"
gum.config["notebook", "graph_rankdir"] = "TB"


## 2. Data Loading and Preparation

Set `PROJECT_ROOT` so that `PROJECT_ROOT / data` holds the parquet exports required for the analysis, or run `projects/HBC/preprocess_data.py` after configuring `config/data_paths.yml` to recreate them.

This notebook expects three inputs:
- `df.parquet`: raw cohort data without imputation (used for comparison plots).
- `df_imp.parquet`: fully imputed dataset feeding the Bayesian network learner.
- `bn_vars.parquet`: metadata table with at least `LAYER` and `VARIABLE NAME` columns describing the expert-assigned layer of each variable.

Update the configuration cell below if your repository layout differs.

In [ ]:

import os
from pathlib import Path

PROJECT_ROOT = Path.cwd()


def _get_path_override(key: str, project_root: Path) -> Path | None:
    """Return path override from env var or .env without hard-coding paths."""
    env_value = os.environ.get(key)
    if env_value:
        return Path(env_value).expanduser()

    env_path = project_root / '.env'
    if not env_path.exists():
        return None

    for raw_line in env_path.read_text().splitlines():
        line = raw_line.strip()
        if not line or line.startswith('#') or '=' not in line:
            continue
        name, value = line.split('=', 1)
        if name.strip() == key:
            cleaned = value.strip()
            if cleaned[:1] in {'"', "'"} and cleaned[-1:] == cleaned[:1]:
                cleaned = cleaned[1:-1]
            if cleaned:
                return Path(cleaned).expanduser()
    return None


candidate_dirs = []

override_path = _get_path_override('BAYESIAN_DATA_DIR', PROJECT_ROOT)
if override_path:
    if not override_path.exists():
        raise FileNotFoundError(
            f"BAYESIAN_DATA_DIR={str(override_path)!r} does not exist. "
            "Update the override value or remove it."
        )
    candidate_dirs.append(override_path)

candidate_dirs += [
    PROJECT_ROOT / 'src' / 'out',
    PROJECT_ROOT / 'out',
    PROJECT_ROOT / 'data',
]

for candidate in candidate_dirs:
    if candidate.exists():
        DATA_DIR = candidate
        break
else:
    DATA_DIR = PROJECT_ROOT / 'data'


In [ ]:
# Load harmonised datasets used throughout the analysis
df = pd.read_parquet(DATA_DIR / 'df.parquet', engine='pyarrow')
df_imp = pd.read_parquet(DATA_DIR / 'df_imp.parquet', engine='pyarrow')
bn_vars = pd.read_parquet(DATA_DIR / 'bn_vars.parquet', engine='pyarrow')

In [ ]:
df_imp.head()

In [ ]:
# Harmonise categorical entries before discretisation
for frame in (df, df_imp):
    # Replace blank or missing stroke history entries with explicit 'No'
    if 'STROKE HISTORY' in frame.columns:
        frame['STROKE HISTORY'] = frame['STROKE HISTORY'].replace({'': 'No', ' ': 'No'}).fillna('No')
    # Legacy CVA column cleaning (if present)
    if 'CVA' in frame.columns:
        frame['CVA'] = frame['CVA'].replace({'': 'Nee', ' ': 'Nee'}).fillna('Nee')


### 2.2 Metadata sanity check
The `bn_vars` table should map every variable used in the network to its corresponding layer.
If you add new variables, ensure the parquet file lists them with the desired `LAYER` label.

Example `bn_vars.parquet` structure (first columns):

| LAYER | VARIABLE NAME | DESCRIPTION |
| --- | --- | --- |
| L0 – Unmodifiable demographics | AGE | Baseline age in years |
| L0 – Unmodifiable demographics | SEX | Biological sex at baseline |
| L2 – Cardiovascular risk factors | VASCULAR RISK SCORE | SCORE2 cardiovascular risk estimate |
| L4 – Potential disease process markers | PTAU181 | Plasma pTau181 concentration category |
| L6 – Current and previous cardiovascular diagnoses / Vascular interventions | PATIENT GROUP | Diagnostic cohort label |
| L8 – Outcomes | OUTCOME_MACE | Binary indicator of follow-up MACE |

Add extra columns (e.g. `DISPLAY NAME`, `NOTES`) as needed—the loader only requires `LAYER` and `VARIABLE NAME`.

In [ ]:
# Ensure DROPOUT REASON is mapped to the dropout layer
if 'bn_vars' in globals():
    if 'DROPOUT REASON' not in bn_vars['VARIABLE NAME'].values:
        bn_vars = pd.concat([bn_vars, pd.DataFrame([{'LAYER': 'L9 – Dropout', 'VARIABLE NAME': 'DROPOUT REASON'}])], ignore_index=True)
    else:
        bn_vars.loc[bn_vars['VARIABLE NAME'] == 'DROPOUT REASON', 'LAYER'] = 'L9 – Dropout'


In [ ]:
# Preview the first few rows once bn_vars.parquet is loaded
bn_vars.sort_values(['LAYER', 'VARIABLE NAME'])

In [ ]:
# Validate the metadata layout before building the network
expected_columns = {'LAYER', 'VARIABLE NAME'}
missing_cols = expected_columns - set(bn_vars.columns)
if missing_cols:
    raise ValueError(f"bn_vars is missing required columns: {missing_cols}")

print('Layers discovered in bn_vars:', sorted(bn_vars['LAYER'].unique()))
display(bn_vars.head())


## 3. Variable Discretisation

Configure the binning strategy feeding each learner and inspect the resulting template.

In [ ]:
# Configure discretisation defaults for mixed-type clinical variables
type_processor = DiscreteTypeProcessor(
  defaultDiscretizationMethod="quantile", defaultNumberOfBins=4, discretizationThreshold=10
)

In [ ]:
# Inspect the discretised template to document the bins used by the learner
# creating a template explaining the variables proposed by the type_processor. This variables will be used by the learner
template = type_processor.discretizedTemplate(df_imp)
for i, n in template:
  print(f"{n:7} : {template.variable(i)}")

In [ ]:
# Inspect the discretised template to document the bins used by the learner
template = type_processor.discretizedTemplate(df_imp)

def format_label(label):
    parts = label.strip('([)').split(';')
    lower = float(parts[0])
    upper = float(parts[1].replace('[', '').replace(')', ''))

    lower_rounded = round(lower, 1)
    upper_rounded = round(upper, 1)

    return f"({lower_rounded};{upper_rounded}["

for i, n in template:
    var = template.variable(i)
    labels = var.labels()

    # Check if the labels look like intervals
    if all((";" in label and "[" in label) for label in labels):
        formatted_labels = [format_label(label) for label in labels]
        print(f"{n:7} : {formatted_labels}")
    else:
        print(f"{n:7} : {labels}")


In [ ]:
# Optional: uncomment to audit discretisation results
#auditDict = type_processor.audit(df_imp)

#print()
#print("** audit **")
#for var in auditDict:
#  print(f"- {var} : ")
#  for k, v in auditDict[var].items():
#    print(f"    + {k} : {v}")

## 4. Knowledge-Guided Network Definition

Translate the expert-defined layers into a reusable Bayesian network template.

In [ ]:
# Map variables onto expert-defined layers for downstream constraints
layer_map = (
    bn_vars.groupby('LAYER')['VARIABLE NAME']
    .apply(list)
    .to_dict()
)


In [ ]:
# Instantiate the Bayesian network using the discretised template as blueprint
bn = gum.BayesNet("LayeredBN")
node_ids = {}

for i in range(template.size()):
    var = template.variable(i)
    name = var.name()
    node_ids[name] = bn.add(var)  # only add once, from template
    

In [ ]:
# Preserve the expert-defined order of layers when applying constraints
# Make sure keys are in the correct order
layer_keys = list(layer_map.keys())  # use list, not sorted, to preserve domain-defined order

## 5. Constrained Structure Learning

Estimate networks that respect the layer order while testing different outcome configurations.

In [ ]:

# Utility: learn a Bayesian network with optional guided structure constraints
import random
from itertools import product


def configure_guided_structure(learner, layer_map_filtered, variables_to_keep, outcomes, enforce=True):
    """Apply dropout-aware arc constraints when enforce=True."""
    layer_keys = list(layer_map_filtered.keys())
    dropout_layer_keys = [k for k in layer_keys if "Dropout" in k]
    dropout_vars = [var for key in dropout_layer_keys for var in layer_map_filtered[key]]
    true_outcomes = [var for var in outcomes if "DROPOUT" not in var.upper()]

    if not enforce:
        return dropout_vars, true_outcomes

    allowed_arcs = []
    for i, from_key in enumerate(layer_keys):
        from_vars = layer_map_filtered[from_key]
        for to_key in layer_keys[i:]:
            to_vars = layer_map_filtered[to_key]
            for parent, child in product(from_vars, to_vars):
                if child in dropout_vars:
                    if parent in outcomes:
                        allowed_arcs.append((parent, child))
                    continue
                if parent in variables_to_keep and child in variables_to_keep:
                    allowed_arcs.append((parent, child))

    for parent in variables_to_keep:
        for child in variables_to_keep:
            if parent != child and (parent, child) not in allowed_arcs:
                learner.addForbiddenArc(parent, child)

    for parent in true_outcomes:
        for child in true_outcomes:
            if parent != child:
                learner.addForbiddenArc(parent, child)

    return dropout_vars, true_outcomes


def build_bn(df, outcomes, layer_map, type_processor, score='K2', use_tabu=True, random_seed=42, enforce_structure=True):

    random.seed(random_seed)
    np.random.seed(random_seed)

    # Step 0: Exclude 'L4 – Potential disease process markers'
    layer_map_filtered = {k: v for k, v in layer_map.items() if k != 'L4 – Potential disease process markers'}
    excluded_vars = layer_map.get('L4 – Potential disease process markers', [])

    # Step 1: Drop excluded outcome variables and excluded biomarkers
    outcomes_all_layers = [v for k, v in layer_map.items() if "Outcomes" in k or "Dropout" in k]
    outcomes_flat = [var for sublist in outcomes_all_layers for var in sublist]
    drop_outcomes = list(set(outcomes_flat) - set(outcomes))

    df_reduced = df.drop(columns=drop_outcomes + excluded_vars, errors='ignore')
    variables_to_keep = df_reduced.columns.tolist()

    # Step 2: Build template from reduced dataframe
    template_reduced = type_processor.discretizedTemplate(df_reduced)

    # Step 3: Instantiate the learner
    learner = gum.BNLearner(df_reduced, template_reduced)

    if score.upper() == 'K2':
        learner.useScoreK2()
    elif score.upper() == 'BIC':
        learner.useScoreBIC()
        learner.useSmoothingPrior()
    elif score.upper() == "BDEU":
        learner.useScoreBDeu(ess=1)
    else:
        raise ValueError("Unsupported score method: choose 'K2' or 'BIC'")

    if use_tabu:
        learner.useLocalSearchWithTabuList()
    else:
        learner.useGreedyHillClimbing()

    # Step 4: Optionally apply guided structure constraints
    dropout_vars, true_outcomes = configure_guided_structure(
        learner,
        layer_map_filtered,
        variables_to_keep,
        outcomes,
        enforce=enforce_structure,
    )

    learner.setMaxIndegree(5)
    # Step 7: Learn the network
    bn = learner.learnBN()
    # Step 8: Ensure outcomes point to dropout layer when enforcing guidance
    if enforce_structure:
        try:
            dropout_ids = [bn.idFromName(name) for name in dropout_vars if name in bn.names()]
            outcome_ids = [bn.idFromName(name) for name in true_outcomes if name in bn.names()]
            for drop_id in dropout_ids:
                for out_id in outcome_ids:
                    if not bn.existsArc(out_id, drop_id):
                        bn.addArc(out_id, drop_id)
        except gum.InvalidArgument:
            pass

    return bn


In [ ]:
bn_cdr = build_bn(df_imp, outcomes=["OUTCOME_CDR_INCREASE", "DROPOUT REASON"], layer_map=layer_map, type_processor=type_processor, score="K2")
bn_mace = build_bn(df_imp, outcomes=["OUTCOME_MACE", "DROPOUT REASON"], layer_map=layer_map, type_processor=type_processor, score="K2")
bn_joint = build_bn(df_imp, outcomes=["OUTCOME_CDR_INCREASE", "OUTCOME_MACE", "DROPOUT REASON"], layer_map=layer_map, type_processor=type_processor, score="K2")

In [ ]:
# Define your ordered layers and assign them increasing color intensity
layer_order = [
    'L0 – Unmodifiable demographics',
    'L1 – Modifiable demograhpics / Lifestyle factors',
    'L2 – Cardiovascular risk factors',
    'L4 – Potential disease process markers',
    'L5 - Imaging markers of neurovascular damage',
    'L6 – Current and previous cardiovascular diagnoses / Vascular interventions',
    'L7 - Functional status',
    'L8 – Outcomes',
    'L9 – Dropout'
]

# Set adjusted min and max color values
min_color_val = 0.111111
max_color_val = 0.99999
steps = len(layer_order) - 1

# Create adjusted color intensity map
layer_color_map = {
    name: min_color_val + i * (max_color_val - min_color_val) / steps
    for i, name in enumerate(layer_order)
}

# Build the nodeColor dictionary
node_colors = {}
for layer_name, variables in layer_map.items():
    color_val = layer_color_map.get(layer_name, 0.5)  # fallback if unknown
    for var in variables:
        if var in bn_joint.names():
            node_colors[var] = color_val

In [ ]:
# Bootstrap helper: quantify edge stability across resampled datasets
from collections import defaultdict
def bootstrap_edge_frequencies(df, outcomes, layer_map, type_processor,
                                build_bn_func, n_bootstraps=100, score='K2', use_tabu=True, random_seed=42):
    edge_counts = defaultdict(int)

    for i in range(n_bootstraps):
        seed = random_seed + i
        random.seed(seed)
        np.random.seed(seed)

        # Bootstrap resample
        boot_df = df.sample(frac=1.0, replace=True, random_state=seed)

        try:
            bn = build_bn_func(boot_df, outcomes, layer_map, type_processor,
                               score=score, use_tabu=use_tabu, random_seed=seed)
        except Exception as e:
            print(f"Bootstrap {i} failed: {e}")
            continue

        # Count arcs
        for parent_id, child_id in bn.arcs():
            parent = bn.variable(parent_id).name()
            child = bn.variable(child_id).name()
            edge_counts[(parent, child)] += 1

    # Normalize to frequencies
    edge_frequencies = {edge: count / n_bootstraps for edge, count in edge_counts.items()}
    return edge_frequencies

In [ ]:
edge_freqs = bootstrap_edge_frequencies(
    df=df_imp,
    outcomes=["OUTCOME_CDR_INCREASE", "OUTCOME_MACE", "DROPOUT REASON"],
    layer_map=layer_map,
    type_processor=type_processor,
    build_bn_func=build_bn,
    n_bootstraps=200,
    score='K2',
    use_tabu=True,
    random_seed=42
)

# Print top edges
for edge, freq in sorted(edge_freqs.items(), key=lambda x: -x[1]):
    print(f"{edge[0]} → {edge[1]}: {freq:.2f}")

In [ ]:
# Get the labels for CDR_INCR
cdr_labels = list(bn_joint.variable(bn_joint.idFromName("OUTCOME_CDR_INCREASE")).labels())
mace_labels = list(bn_joint.variable(bn_joint.idFromName("OUTCOME_MACE")).labels())

## 6. Posterior Inference and Edge Stability

Summarise how the learned networks behave under different evidence configurations and assess structural robustness.

### 6.1 Posterior inference for CDR

Visualise how upstream factors shift when conditioning on `OUTCOME_CDR_INCREASE`.

Condition the CDR network on each outcome modality to review the upstream clinical drivers.

In [ ]:
gnb.sideBySide(
    *[
        gnb.showBN(bn_joint, size="9", nodeColor=node_colors, cmapNode=plt.get_cmap("coolwarm"))
    ],
    captions=[f"Inference given that CDR_INCREASE = {label}" for label in cdr_labels],
    ncols=1
)

### 6.2 Joint inference for CDR and MACE

Inspect posterior behaviour when conditioning on both outcome nodes simultaneously.

Focus on the parents of each outcome to summarise the conditional probability tables.

In [ ]:
# Create inference engine
ie = gum.LazyPropagation(bn_joint)
ie.makeInference()

# Compute MI and apply it to arcs using variable indices (not names)
arc_width = {}

for u, v in bn_joint.arcs():  # u and v are variable IDs
    try:
        it = gum.InformationTheory(ie, v, [u])  # use IDs directly
        mi = it.mutualInformationXY()
    except:
        mi = 0.1

    arc_width[(u, v)] = mi  # ✅ use indices, not names

max_mi = max(arc_width.values())
arc_width = {k: (np.sqrt(v) / np.sqrt(max_mi) * 3.0 + 0.5) for k, v in arc_width.items()}


In [ ]:
var_name_to_id = {bn_joint.variable(i).name(): i for i in range(bn_joint.size())}

bootstrap_freq_id = {}

for (parent_name, child_name), freq in edge_freqs.items():
    try:
        parent_id = var_name_to_id[parent_name]
        child_id = var_name_to_id[child_name]
        bootstrap_freq_id[(parent_id, child_id)] = freq
    except KeyError:
        continue  # skip if variable not found in network

In [ ]:
arc_label = {
    arc: f"f={int(bootstrap_freq_id.get(arc, 0.0) * 100)}%"
    for arc in bn_joint.arcs()
}

In [ ]:
arc_label

In [ ]:
arc_color_values = {
    arc: bootstrap_freq_id.get(arc, 0.0)
    for arc in bn_joint.arcs()
}


In [ ]:
# Plot with MI-based edge widths
gnb.showBN(
    bn_joint,
    arcWidth=arc_width,
    nodeColor=node_colors,
    arcColor=arc_color_values,
    arcLabel=arc_label,
    cmapNode=plt.get_cmap("coolwarm"),
    cmapArc=plt.get_cmap("Blues")
)

In [ ]:
df_imp.head()

In [ ]:
# Plot with MI-based edge widths
gnb.showInference(
    bn_joint,
    arcWidth=arc_width,
    nodeColor=node_colors,
    arcColor=arc_color_values,
    cmapNode=plt.get_cmap("coolwarm"),
    cmapArc=plt.get_cmap("Blues")
)

Here, the whole network is considered

In [ ]:
gnb.sideBySide(
  "<H3>OUTCOME CDR INCREASE</H3>",
  "<H3>OUTCOME MACE</H3>",
  bn_joint.cpt("OUTCOME_CDR_INCREASE"),
  bn_joint.cpt("OUTCOME_MACE"),
  ncols=2,
)

### 6.3 Uncorrected mutual information — CDR_INCREASE

In [ ]:
# Initialize inference engine
ie = gum.LazyPropagation(bn_joint)

# Get all variables in the network except the outcome
all_variables = bn_joint.names()
variables_to_test = [var for var in all_variables if var != 'OUTCOME_CDR_INCREASE']

# Store results
mi_results = {}

for var in variables_to_test:
    try:
        it = gum.InformationTheory(ie, 'OUTCOME_CDR_INCREASE', [var])
        mi = it.mutualInformationXY()  # Without correction
        mi_results[var] = mi
        print(f'Mutual Information ({var} -> OUTCOME_CDR_INCREASE): {mi:.6f}')
    except Exception as e:
        print(f'Error processing variable {var}: {e}')

# Optional: sort results
sorted_mi = sorted(mi_results.items(), key=lambda x: x[1], reverse=True)

print("\nVariable Importance Ranking for OUTCOME_CDR_INCREASE (Raw Mutual Information)")
for var, score in sorted_mi:
    print(f'{var}: {score:.6f}')

### 6.4 Corrected mutual information — CDR_INCREASE

In [ ]:
# Initialize inference engine
ie = gum.LazyPropagation(bn_joint)

# Get the parents of OUTCOME_CDR_INCREASE
parents = bn_joint.parents(bn_joint.idFromName('OUTCOME_CDR_INCREASE'))
parent_names = [bn_joint.variable(p).name() for p in parents]

# Get all variables in the network except OUTCOME_CDR_INCREASE and its parents
all_variables = bn_joint.names()
variables_to_test = [var for var in all_variables if var != 'OUTCOME_CDR_INCREASE' and var not in parent_names]

# Initialize dictionary to store results
cmi_results = {}

# Loop through each variable and compute conditional mutual information
for var in variables_to_test:
    try:
        it = gum.InformationTheory(ie, 'OUTCOME_CDR_INCREASE', [var], parent_names)
        cmi = it.mutualInformationXYgivenZ()
        cmi_results[var] = cmi
        print(f'Corrected MI ({var} -> OUTCOME_CDR_INCREASE | {parent_names}): {cmi:.6f}')
    except Exception as e:
        print(f'Error processing variable {var}: {e}')

# Optional: sort and display ranking
sorted_cmi = sorted(cmi_results.items(), key=lambda x: x[1], reverse=True)

print("\nVariable Importance Ranking for OUTCOME_CDR_INCREASE (Corrected Mutual Information)")
for var, score in sorted_cmi:
    print(f'{var}: {score:.6f}')

### 6.5 Uncorrected mutual information — EVENT_MACE

In [ ]:
# Initialize inference engine
ie = gum.LazyPropagation(bn_joint)

# Get all variables in the network except the outcome
all_variables = bn_joint.names()
variables_to_test = [var for var in all_variables if var != 'OUTCOME_MACE']

# Store results
mi_results = {}

for var in variables_to_test:
    try:
        it = gum.InformationTheory(ie, 'OUTCOME_MACE', [var])
        mi = it.mutualInformationXY()  # Without correction
        mi_results[var] = mi
        print(f'Mutual Information ({var} -> OUTCOME_MACE): {mi:.6f}')
    except Exception as e:
        print(f'Error processing variable {var}: {e}')

# Optional: sort results
sorted_mi = sorted(mi_results.items(), key=lambda x: x[1], reverse=True)

print("\nVariable Importance Ranking for OUTCOME_MACE (Raw Mutual Information)")
for var, score in sorted_mi:
    print(f'{var}: {score:.6f}')

### 6.6 Corrected mutual information — EVENT_MACE

In [ ]:
# Initialize inference engine
ie = gum.LazyPropagation(bn_joint)

# Get the parents of EVENT_MACE
parents = bn_joint.parents(bn_joint.idFromName('OUTCOME_MACE'))
parent_names = [bn_joint.variable(p).name() for p in parents]

# Get all variables in the network except OUTCOME_MACE and its parents
all_variables = bn_joint.names()
variables_to_test = [var for var in all_variables if var != 'OUTCOME_MACE' and var not in parent_names]

# Initialize dictionary to store results
cmi_results = {}

# Loop through each variable and compute conditional mutual information
for var in variables_to_test:
    try:
        it = gum.InformationTheory(ie, 'OUTCOME_MACE', [var], parent_names)
        cmi = it.mutualInformationXYgivenZ()
        cmi_results[var] = cmi
        print(f'Corrected MI ({var} -> OUTCOME_MACE | {parent_names}): {cmi:.6f}')
    except Exception as e:
        print(f'Error processing variable {var}: {e}')

# Optional: sort and display ranking
sorted_cmi = sorted(cmi_results.items(), key=lambda x: x[1], reverse=True)

print("\nVariable Importance Ranking for OUTCOME_MACE (Corrected Mutual Information)")
for var, score in sorted_cmi:
    print(f'{var}: {score:.6f}')

## 7. Biomarker Layer Ablation

Contrast networks learned with and without the specialised biomarker layer.

In [ ]:
import random
from itertools import product

def build_bn_biomarkers(df, outcomes, layer_map, type_processor, score='K2', use_tabu=True, random_seed=42):
    
    random.seed(random_seed)
    np.random.seed(random_seed)

    # Step 1: Drop excluded outcome variables and excluded biomarkers
    outcomes_all_layers = [v for k, v in layer_map.items() if "Outcomes" in k or "Dropout" in k]
    outcomes_flat = [var for sublist in outcomes_all_layers for var in sublist]
    drop_outcomes = list(set(outcomes_flat) - set(outcomes))

    df_reduced = df.drop(columns=drop_outcomes, errors='ignore')
    variables_to_keep = df_reduced.columns.tolist()

    # Step 2: Build template from reduced dataframe
    template_reduced = type_processor.discretizedTemplate(df_reduced)

    # Step 3: Instantiate the learner
    learner = gum.BNLearner(df_reduced, template_reduced)

    if score.upper() == 'K2':
        learner.useScoreK2()
    elif score.upper() == 'BIC':
        learner.useScoreBIC()
        learner.useSmoothingPrior()
    elif score.upper() == "BDEU":
        learner.useScoreBDeu(ess=1)
    else:
        raise ValueError("Unsupported score method: choose 'K2' or 'BIC'")

    if use_tabu:
        learner.useLocalSearchWithTabuList()
    else:
        learner.useGreedyHillClimbing()

  # Step 4: Build allowed arcs with special handling for dropout layer
    layer_keys = list(layer_map.keys())

    dropout_layer_keys = [k for k in layer_keys if "Dropout" in k]
    dropout_vars = []
    for key in dropout_layer_keys:
        dropout_vars += layer_map[key]

    allowed_arcs = []

    for i in range(len(layer_keys)):
        from_vars = layer_map[layer_keys[i]]
        for j in range(i, len(layer_keys)):
            to_vars = layer_map[layer_keys[j]]

            for parent, child in product(from_vars, to_vars):
                # Special rule: only allow arcs into dropout vars if from outcome
                if child in dropout_vars:
                    if parent in outcomes:
                        allowed_arcs.append((parent, child))
                    continue  # skip all others going into dropout

                if parent in variables_to_keep and child in variables_to_keep:
                    allowed_arcs.append((parent, child))

    # Step 5: Enforce allowed arcs, forbid the rest
    for parent in variables_to_keep:
        for child in variables_to_keep:
            if parent != child and (parent, child) not in allowed_arcs:
                learner.addForbiddenArc(parent, child)

    # Step 6: Disallow arcs between actual outcomes, but allow outcomes → dropout
    true_outcomes = [var for var in outcomes if "DROPOUT" not in var.upper()]
    for parent in true_outcomes:
        for child in true_outcomes:
            if parent != child:
                learner.addForbiddenArc(parent, child)

    learner.setMaxIndegree(5)
    # Step 7: Learn the network
    bn = learner.learnBN()
    return bn

In [ ]:
bn_with_biomarkers = build_bn_biomarkers(df_imp, ["OUTCOME_CDR_INCREASE", "OUTCOME_MACE", "DROPOUT REASON"], layer_map=layer_map, type_processor=type_processor, score="K2")

### 7.1 Biomarker-outcome logistic regression

Quantify adjusted associations between each biomarker and the clinical outcomes using logistic regression models with age and sex as covariates.

In [ ]:
import statsmodels.formula.api as smf

biomarker_vars = [
    var for var in layer_map.get('L4 – Potential disease process markers', [])
    if var in df_imp.columns
]
outcome_labels = {
    'OUTCOME_MACE': 'MACE event',
    'OUTCOME_CDR_INCREASE': 'CDR increase'
}

logit_rows = []

for outcome, label in outcome_labels.items():
    observed = df_imp[df_imp[outcome].isin(['Yes', 'No'])].copy()
    if observed.empty:
        continue

    outcome_bin = f"{outcome}_bin"
    observed[outcome_bin] = (observed[outcome] == 'Yes').astype(int)

    for biomarker in biomarker_vars:
        biomarker_term = f'Q("{biomarker}")'
        model_specs = [
            {
                "label": "Unadjusted",
                "required_vars": [outcome_bin, biomarker],
                "formula_suffix": ""
            },
            {
                "label": "Adjusted (age + sex)",
                "required_vars": [outcome_bin, biomarker, 'AGE', 'SEX'],
                "formula_suffix": " + AGE + C(SEX)"
            },
        ]

        for spec in model_specs:
            missing_covars = [col for col in spec["required_vars"] if col not in observed.columns]
            if missing_covars:
                logit_rows.append({
                    'Outcome': label,
                    'Biomarker': biomarker,
                    'Model': spec["label"],
                    'Odds Ratio': np.nan,
                    'CI 2.5%': np.nan,
                    'CI 97.5%': np.nan,
                    'p-value': np.nan,
                    'N': 0,
                    'Notes': f"Missing covariates: {', '.join(missing_covars)}"
                })
                continue

            model_df = observed[spec["required_vars"]].dropna()
            sample_size = int(model_df.shape[0])

            if sample_size == 0:
                logit_rows.append({
                    'Outcome': label,
                    'Biomarker': biomarker,
                    'Model': spec["label"],
                    'Odds Ratio': np.nan,
                    'CI 2.5%': np.nan,
                    'CI 97.5%': np.nan,
                    'p-value': np.nan,
                    'N': sample_size,
                    'Notes': 'No complete cases after filtering'
                })
                continue

            if model_df[outcome_bin].nunique() < 2 or model_df[biomarker].nunique() < 2:
                logit_rows.append({
                    'Outcome': label,
                    'Biomarker': biomarker,
                    'Model': spec["label"],
                    'Odds Ratio': np.nan,
                    'CI 2.5%': np.nan,
                    'CI 97.5%': np.nan,
                    'p-value': np.nan,
                    'N': sample_size,
                    'Notes': 'Insufficient variation'
                })
                continue

            suffix = spec["formula_suffix"]
            formula = f"{outcome_bin} ~ {biomarker_term}{suffix}"

            try:
                fit = smf.logit(formula=formula, data=model_df).fit(disp=0)
            except Exception as exc:
                logit_rows.append({
                    'Outcome': label,
                    'Biomarker': biomarker,
                    'Model': spec["label"],
                    'Odds Ratio': np.nan,
                    'CI 2.5%': np.nan,
                    'CI 97.5%': np.nan,
                    'p-value': np.nan,
                    'N': sample_size,
                    'Notes': f'Fit failed: {exc}'
                })
                continue

            param_name = biomarker_term

            if param_name not in fit.params.index:
                logit_rows.append({
                    'Outcome': label,
                    'Biomarker': biomarker,
                    'Model': spec["label"],
                    'Odds Ratio': np.nan,
                    'CI 2.5%': np.nan,
                    'CI 97.5%': np.nan,
                    'p-value': np.nan,
                    'N': sample_size,
                    'Notes': 'Biomarker term dropped during fitting'
                })
                continue

            conf = fit.conf_int().loc[param_name]
            logit_rows.append({
                'Outcome': label,
                'Biomarker': biomarker,
                'Model': spec["label"],
                'Odds Ratio': np.exp(fit.params[param_name]),
                'CI 2.5%': np.exp(conf[0]),
                'CI 97.5%': np.exp(conf[1]),
                'p-value': fit.pvalues[param_name],
                'N': sample_size,
                'Notes': ''
            })

logit_results = pd.DataFrame(logit_rows)

if not logit_results.empty:
    display_order = ['Unadjusted', 'Adjusted (age + sex)']
    formatted = logit_results.copy()
    formatted['Model'] = pd.Categorical(formatted['Model'], categories=display_order, ordered=True)
    formatted['Odds Ratio (95% CI)'] = formatted.apply(
        lambda row: (
            f"{row['Odds Ratio']:.2f} ({row['CI 2.5%']:.2f}, {row['CI 97.5%']:.2f})"
            if pd.notnull(row['Odds Ratio']) else 'NA'
        ),
        axis=1
    )
    formatted['p-value'] = formatted['p-value'].map(
        lambda x: f"{x:.3g}" if pd.notnull(x) else 'NA'
    )
    formatted = formatted.sort_values(['Outcome', 'Biomarker', 'Model']).reset_index(drop=True)

    wide = (
        formatted
        .pivot_table(
            index=['Outcome', 'Biomarker', 'N'],
            columns='Model',
            values=['Odds Ratio (95% CI)', 'p-value'],
            aggfunc='first'
        )
        .sort_index()
    )

    wide.columns = [f"{metric} ({model})" for metric, model in wide.columns]
    wide = wide.reset_index()
    display(wide[['Outcome', 'Biomarker', 'N'] + [col for col in wide.columns if col not in {'Outcome', 'Biomarker', 'N'}]])
else:
    display(pd.DataFrame({'Message': ['No logistic regression models were fitted.']}))


#### Biomarker distribution checks

Visualise biomarker levels by CDR outcome status to verify the magnitude of group differences suggested by the logistic regression.

In [ ]:
# Create inference engine
ie = gum.LazyPropagation(bn_with_biomarkers)
ie.makeInference()

# Compute MI and apply it to arcs using variable indices (not names)
arc_width = {}

for u, v in bn_with_biomarkers.arcs():  # u and v are variable IDs
    try:
        it = gum.InformationTheory(ie, v, [u])  # use IDs directly
        mi = it.mutualInformationXY()
    except:
        mi = 0.1

    arc_width[(u, v)] = mi  # ✅ use indices, not names

max_mi = max(arc_width.values())
arc_width = {k: (np.sqrt(v) / np.sqrt(max_mi) * 3.0 + 0.5) for k, v in arc_width.items()}

In [ ]:
var_name_to_id = {bn_with_biomarkers.variable(i).name(): i for i in range(bn_with_biomarkers.size())}

bootstrap_freq_id = {}

for (parent_name, child_name), freq in edge_freqs.items():
    try:
        parent_id = var_name_to_id[parent_name]
        child_id = var_name_to_id[child_name]
        bootstrap_freq_id[(parent_id, child_id)] = freq
    except KeyError:
        continue  # skip if variable not found in network

In [ ]:
# Update layer_order for color mapping and arc logic
layer_order = [
    'L0 – Unmodifiable demographics',
    'L1 – Modifiable demograhpics / Lifestyle factors',
    'L2 – Cardiovascular risk factors',
    'L3 – Specialized biomarkers',  # <-- NEW LAYER
    'L4 – Potential disease process markers',
    'L5 - Imaging markers of neurovascular damage',
    'L6 – Current and previous cardiovascular diagnoses / Vascular interventions',
    'L7 - Functional status',
    'L8 – Outcomes',
    'L9 – Dropout'
]

In [ ]:
# Set adjusted min and max color values
min_color_val = 0.111111
max_color_val = 0.99999
steps = len(layer_order) - 1

# Create adjusted color intensity map
layer_color_map = {
    name: min_color_val + i * (max_color_val - min_color_val) / steps
    for i, name in enumerate(layer_order)
}

# Build the nodeColor dictionary
node_colors = {}
for layer_name, variables in layer_map.items():
    color_val = layer_color_map.get(layer_name, 0.5)  # fallback if unknown
    for var in variables:
        if var in bn_with_biomarkers.names():
            node_colors[var] = color_val

In [ ]:
from collections import defaultdict
def bootstrap_edge_frequencies(df, outcomes, layer_map, type_processor,
                                build_bn_func, n_bootstraps=100, score='K2', use_tabu=True, random_seed=42):
    edge_counts = defaultdict(int)

    for i in range(n_bootstraps):
        seed = random_seed + i
        random.seed(seed)
        np.random.seed(seed)

        # Bootstrap resample
        boot_df = df.sample(frac=1.0, replace=True, random_state=seed)

        try:
            bn = build_bn_func(boot_df, outcomes, layer_map, type_processor,
                               score=score, use_tabu=use_tabu, random_seed=seed)
        except Exception as e:
            print(f"Bootstrap {i} failed: {e}")
            continue

        # Count arcs
        for parent_id, child_id in bn.arcs():
            parent = bn.variable(parent_id).name()
            child = bn.variable(child_id).name()
            edge_counts[(parent, child)] += 1

    # Normalize to frequencies
    edge_frequencies = {edge: count / n_bootstraps for edge, count in edge_counts.items()}
    return edge_frequencies

In [ ]:
# Met biomarkers
edge_freqs_with = bootstrap_edge_frequencies(
    df=df_imp,
    outcomes=["OUTCOME_CDR_INCREASE", "OUTCOME_MACE", "DROPOUT REASON"],
    layer_map=layer_map,
    type_processor=type_processor,
    build_bn_func=build_bn_biomarkers,
    n_bootstraps=10,
    score='K2',
    use_tabu=True

)

# Print top edges
for edge, freq in sorted(edge_freqs_with.items(), key=lambda x: -x[1]):
    print(f"{edge[0]} → {edge[1]}: {freq:.2f}")

In [ ]:
# Create inference engine
ie = gum.LazyPropagation(bn_with_biomarkers)
ie.makeInference()

# Compute MI and apply it to arcs using variable indices (not names)
arc_width = {}

for u, v in bn_with_biomarkers.arcs():  # u and v are variable IDs
    try:
        it = gum.InformationTheory(ie, v, [u])  # use IDs directly
        mi = it.mutualInformationXY()
    except:
        mi = 0.1

    arc_width[(u, v)] = mi  # ✅ use indices, not names

max_mi = max(arc_width.values())
arc_width = {k: (np.sqrt(v) / np.sqrt(max_mi) * 3.0 + 0.5) for k, v in arc_width.items()}

In [ ]:
# Use edge_freqs_with for the network with biomarkers!
var_name_to_id = {bn_with_biomarkers.variable(i).name(): i for i in range(bn_with_biomarkers.size())}

bootstrap_freq_id = {}

for (parent_name, child_name), freq in edge_freqs_with.items():  # <-- use edge_freqs_with here!
    try:
        parent_id = var_name_to_id[parent_name]
        child_id = var_name_to_id[child_name]
        bootstrap_freq_id[(parent_id, child_id)] = freq
    except KeyError:
        continue  # skip if variable not found in network

arc_color_values = {
    arc: bootstrap_freq_id.get(arc, 0.0)
    for arc in bn_with_biomarkers.arcs()
}

In [ ]:
# Plot with MI-based edge widths
gnb.showInference(
    bn_with_biomarkers,
    arcWidth=arc_width,
    nodeColor=node_colors,
    arcColor=arc_color_values,
    cmapArc=plt.get_cmap("Blues"),
    cmapNode=plt.get_cmap("coolwarm")
    )

In [ ]:
arc_label = {
    arc: f"f={int(bootstrap_freq_id.get(arc, 0.0) * 100)}%"
    for arc in bn_with_biomarkers.arcs()
}

In [ ]:
# Plot with MI-based edge widths
gnb.showBN(
    bn_with_biomarkers,
    arcWidth=arc_width,
    nodeColor=node_colors,
    arcColor=arc_color_values,
    arcLabel=arc_label,
    cmapNode=plt.get_cmap("coolwarm"),
    cmapArc=plt.get_cmap("Blues")
)

The specialised biomarkers appear upstream but are not directly connected to the outcomes; removing the layer leaves predictive behaviour largely unchanged.

In [ ]:
gnb.showTensor(bn_with_biomarkers.cpt(bn_with_biomarkers.idFromName("OUTCOME_CDR_INCREASE")), digits=2)

In [ ]:
gnb.showTensor(bn_with_biomarkers.cpt(bn_with_biomarkers.idFromName("OUTCOME_MACE")), digits=2)

In [ ]:
# Initialize inference engine
ie = gum.LazyPropagation(bn_with_biomarkers)

# Get all variables in the network except the outcome
all_variables = bn_with_biomarkers.names()
variables_to_test = [var for var in all_variables if var != 'OUTCOME_MACE']

# Store results
mi_results = {}

for var in variables_to_test:
    try:
        it = gum.InformationTheory(ie, 'OUTCOME_MACE', [var])
        mi = it.mutualInformationXY()  # Without correction
        mi_results[var] = mi
        print(f'Mutual Information ({var} -> OUTCOME_MACE): {mi:.6f}')
    except Exception as e:
        print(f'Error processing variable {var}: {e}')

# Optional: sort results
sorted_mi = sorted(mi_results.items(), key=lambda x: x[1], reverse=True)

print("\nVariable Importance Ranking for OUTCOME_MACE (Raw Mutual Information)")
for var, score in sorted_mi:
    print(f'{var}: {score:.6f}')

In [ ]:
# Initialize inference engine
ie = gum.LazyPropagation(bn_with_biomarkers)

# Get all variables in the network except the outcome
all_variables = bn_with_biomarkers.names()
variables_to_test = [var for var in all_variables if var != 'OUTCOME_CDR_INCREASE']

# Store results
mi_results = {}

for var in variables_to_test:
    try:
        it = gum.InformationTheory(ie, 'OUTCOME_CDR_INCREASE', [var])
        mi = it.mutualInformationXY()  # Without correction
        mi_results[var] = mi
        print(f'Mutual Information ({var} -> OUTCOME_CDR_INCREASE): {mi:.6f}')
    except Exception as e:
        print(f'Error processing variable {var}: {e}')

# Optional: sort results
sorted_mi = sorted(mi_results.items(), key=lambda x: x[1], reverse=True)

print("\nVariable Importance Ranking for OUTCOME_CDR_INCREASE (Raw Mutual Information)")
for var, score in sorted_mi:
    print(f'{var}: {score:.6f}')

In [ ]:
# Initialize inference engine
ie = gum.LazyPropagation(bn_with_biomarkers)

# Get the parents of OUTCOME_CDR_INCREASE
parents = bn_with_biomarkers.parents(bn_with_biomarkers.idFromName('OUTCOME_CDR_INCREASE'))
parent_names = [bn_with_biomarkers.variable(p).name() for p in parents]

# Get all variables in the network except OUTCOME_CDR_INCREASE and its parents
all_variables = bn_with_biomarkers.names()
variables_to_test = [var for var in all_variables if var != 'OUTCOME_CDR_INCREASE' and var not in parent_names]

# Initialize dictionary to store results
cmi_results = {}

# Loop through each variable and compute conditional mutual information
for var in variables_to_test:
    try:
        it = gum.InformationTheory(ie, 'OUTCOME_CDR_INCREASE', [var], parent_names)
        cmi = it.mutualInformationXYgivenZ()
        cmi_results[var] = cmi
        print(f'Corrected MI ({var} -> OUTCOME_CDR_INCREASE | {parent_names}): {cmi:.6f}')
    except Exception as e:
        print(f'Error processing variable {var}: {e}')

# Optional: sort and display ranking
sorted_cmi = sorted(cmi_results.items(), key=lambda x: x[1], reverse=True)

print("\nVariable Importance Ranking for OUTCOME_CDR_INCREASE (Corrected Mutual Information)")
for var, score in sorted_cmi:
    print(f'{var}: {score:.6f}')

## 8. Scenario Exploration

Explore illustrative patient trajectories via direct inference and narrative summaries.

### 8.1 Bootstrapped outcome risks

Quantify uncertainty around the scenario-level outcome probabilities by refitting the joint network on bootstrap resamples and re-running inference for each patient profile.

In [ ]:
from collections import defaultdict

def bootstrap_scenario_risks(
    df,
    scenario_profiles,
    target_outcomes,
    build_bn_func,
    layer_map,
    type_processor,
    n_bootstraps=200,
    score='K2',
    use_tabu=True,
    random_seed=42,
):
    samples = defaultdict(list)
    outcome_state_labels = {}
    failed = 0

    for i in range(n_bootstraps):
        seed = random_seed + i
        boot_df = df.sample(frac=1.0, replace=True, random_state=seed)
        try:
            bn_iter = build_bn_func(
                boot_df,
                outcomes=['OUTCOME_CDR_INCREASE', 'OUTCOME_MACE', 'DROPOUT REASON'],
                layer_map=layer_map,
                type_processor=type_processor,
                score=score,
                use_tabu=use_tabu,
                random_seed=seed,
            )
        except Exception:
            failed += 1
            continue

        for scenario_name, evidence in scenario_profiles:
            ie = gum.LazyPropagation(bn_iter)
            try:
                ie.setEvidence(evidence)
            except gum.InvalidArgument:
                continue

            for outcome_name, outcome_label in target_outcomes:
                if outcome_name not in outcome_state_labels:
                    var = bn_iter.variable(bn_iter.idFromName(outcome_name))
                    outcome_state_labels[outcome_name] = list(var.labels())
                posterior = ie.posterior(outcome_name)
                for idx_state, state_label in enumerate(outcome_state_labels[outcome_name]):
                    samples[(scenario_name, outcome_name, state_label)].append(float(posterior[idx_state]))

    rows = []
    for (scenario_name, outcome_name, state_label), values in samples.items():
        values = np.asarray(values)
        values = values[np.isfinite(values)]
        if values.size == 0:
            continue
        rows.append(
            {
                'Scenario': scenario_name,
                'Outcome': dict(target_outcomes).get(outcome_name, outcome_name),
                'Category': state_label,
                'Mean probability': values.mean(),
                'Std': values.std(ddof=1) if values.size > 1 else 0.0,
                'CI 2.5%': np.quantile(values, 0.025),
                'CI 97.5%': np.quantile(values, 0.975),
                'Bootstraps': int(values.size),
            }
        )

    if failed:
        print(f"Warning: {failed} bootstrap fits failed and were skipped.")
    return pd.DataFrame(rows).sort_values(['Scenario', 'Outcome', 'Category']).reset_index(drop=True)

healthy_evs = {
    'PATIENT GROUP': 'Reference',
    'SMALL VESSEL DISEASE SCORE': '0',
    'AGE': '58',
    'MINI MENTAL STATE EXAMINATION': '29',
    'BASELINE CDR': '0',
    'SEX': 'Male',
}

ill_evs = {
    'PATIENT GROUP': 'Vascular cognitive impairment',
    'SMALL VESSEL DISEASE SCORE': '2',
    'AGE': '76',
    'BASELINE CDR': '0',
    'SEX': 'Female',
}

scenario_profiles = [
    ('Healthy profile', healthy_evs),
    ('Ill profile', ill_evs),
]

target_outcomes = [
    ('OUTCOME_CDR_INCREASE', 'CDR increase'),
    ('OUTCOME_MACE', 'MACE event'),
]

scenario_bootstrap_summary = bootstrap_scenario_risks(
    df=df_imp,
    scenario_profiles=scenario_profiles,
    target_outcomes=target_outcomes,
    build_bn_func=build_bn,
    layer_map=layer_map,
    type_processor=type_processor,
    n_bootstraps=200,
    score='K2',
    use_tabu=True,
    random_seed=42,
)

display(scenario_bootstrap_summary)


In [ ]:
ie = gum.LazyPropagation(bn_joint)

# Healthy profile
healthy_evs = {
    'PATIENT GROUP': 'Reference',
    'SMALL VESSEL DISEASE SCORE': '0',
    'AGE': '58',
    'MINI MENTAL STATE EXAMINATION': '29',
    'BASELINE CDR': '0',
    'SEX': 'Male'
}

# Calculate probabilities for healthy patient
ie.setEvidence(healthy_evs)
healthy_posterior = ie.posterior('OUTCOME_CDR_INCREASE')
print("Healthy Patient - OUTCOME_CDR_INCREASE probabilities:")
healthy_posterior

In [ ]:
# Calculate probabilities for healthy patient
ie.setEvidence(healthy_evs)
healthy_posterior = ie.posterior('OUTCOME_MACE')
print("Healthy Patient - OUTCOME_MACE probabilities:")
healthy_posterior

In [ ]:
gnb.showInference(bn_joint, 
                  evs=healthy_evs, 
                  size="10!", 
                  nodeColor=node_colors, 
                  cmapNode=plt.get_cmap("coolwarm"))

In [ ]:
ie = gum.LazyPropagation(bn_joint)

# Healthy profile
ill_evs = {
    'PATIENT GROUP': 'Vascular cognitive impairment',
    'SMALL VESSEL DISEASE SCORE': '2',
    'AGE': '76',
    'BASELINE CDR': '0',
    'SEX': 'Female'
}


# Calculate probabilities for ill patient
ie.setEvidence(ill_evs)
ill_posterior = ie.posterior('OUTCOME_CDR_INCREASE')
print("Ill Patient - OUTCOME_CDR_INCREASE probabilities:")
ill_posterior

In [ ]:
# Calculate probabilities for ill patient
ie.setEvidence(ill_evs)
ill_posterior = ie.posterior('OUTCOME_MACE')
print("Ill Patient - OUTCOME_MACE probabilities:")
ill_posterior

In [ ]:
gnb.showInference(bn_joint, evs=ill_evs, size="10!", nodeColor=node_colors, cmapNode=plt.get_cmap("coolwarm"))

## 9. Persistence and Next Steps

Export the trained network and outline follow-up visual summaries.

In [ ]:
# Persist the joint network for reuse outside the notebook
gum.saveBN(bn_joint, "bn_joint.bifxml")  # You can also use .net or .xdsl
